In [2]:
import spacy
import json

# Modell laden
nlp = spacy.load("../output/model-best")

# Testdaten laden
with open("../data/real_focus_zusammengefasst/test_zusammengefasst.json", "r", encoding="utf-8") as f:
    test_data = json.load(f)

# Funktion zum Anonymisieren
def anonymize(text, doc):
    """
    Ersetzt erkannte Entitäten durch Platzhalter wie [NAME], [ADRESSE] etc.
    """
    new_text = text
    for ent in sorted(doc.ents, key=lambda e: -e.start_char):  # rückwärts ersetzen
        new_text = new_text[:ent.start_char] + f"[{ent.label_}]" + new_text[ent.end_char:]
    return new_text

# Anwenden und anzeigen
for i, example in enumerate(test_data[:10]):  # ändere die Zahl für mehr Beispiele
    text = example["text"]
    doc = nlp(text)
    anonymized = anonymize(text, doc)

    print(f"\n🔹 Beispiel {i+1}")
    print("📩 Original:")
    print(text.strip())
    print("🕵️‍♀️ Anonymisiert:")
    print(anonymized.strip())
    print("-" * 80)



from collections import defaultdict
from sklearn.metrics import precision_recall_fscore_support

# Modell laden
nlp = spacy.load("../output/model-best")

# Testdaten laden
with open("../data/real_focus_zusammengefasst/test_entities_zusammengefasst.json", "r", encoding="utf-8") as f:
    data = json.load(f)

true_labels = []
pred_labels = []

# Zuordnung für Auswertung pro Label
all_labels = set()
per_label_true = defaultdict(list)
per_label_pred = defaultdict(list)

for example in data:
    text = example["text"]
    gold_ents = {(start, end, label) for start, end, label in example["entities"]}
    doc = nlp(text)
    pred_ents = {(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents}

    labels_in_gold = set(label for _, _, label in gold_ents)
    labels_in_pred = set(label for _, _, label in pred_ents)
    all_labels.update(labels_in_gold)
    all_labels.update(labels_in_pred)

    for label in all_labels:
        gold = {span for span in gold_ents if span[2] == label}
        pred = {span for span in pred_ents if span[2] == label}

        for span in gold | pred:
            per_label_true[label].append(1 if span in gold else 0)
            per_label_pred[label].append(1 if span in pred else 0)

# Ergebnis anzeigen
for label in sorted(all_labels):
    p, r, f1, _ = precision_recall_fscore_support(
        per_label_true[label], per_label_pred[label], average="binary", zero_division=0
    )
    print(f"🔎 Label: {label}")
    print(f"  Precision: {p:.2%}")
    print(f"  Recall:    {r:.2%}")
    print(f"  F1-Score:  {f1:.2%}")
    print("-" * 30)




🔹 Beispiel 1
📩 Original:
Sehr geehrte Damen und Herren,
ich widerrufe hiermit meine Einwilligung in die Verarbeitung meiner
personenbezogenen Daten. Ich fordere Sie darüber hinaus auf, die von Ihnen
zu meiner Person gespeicherten personenbezogenen Daten zu löschen und mir
anschließend die Löschung unverzüglich zu bestätigen.
Sollten Gründe gegen die Löschung meiner personenbezogenen Daten sprechen,
bitte ich um Angaben der jeweiligen gesetzlichen Grundlage.
Mit freundlichen Grüßen
Manuel Krein
🕵️‍♀️ Anonymisiert:
Sehr geehrte Damen und Herren,
ich widerrufe hiermit meine Einwilligung in die Verarbeitung meiner
personenbezogenen Daten. Ich fordere Sie darüber hinaus auf, die von Ihnen
zu meiner Person gespeicherten personenbezogenen Daten zu löschen und mir
anschließend die Löschung unverzüglich zu bestätigen.
Sollten Gründe gegen die Löschung meiner personenbezogenen Daten sprechen,
bitte ich um Angaben der jeweiligen gesetzlichen Grundlage.
Mit freundlichen Grüßen
[NAME]
------------